In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split,train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,mean_squared_error, mean_absolute_error, r2_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [2]:
import boto3
s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/alllabs1000adm/alllabs1000adm.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
rawdata = pd.read_csv(obj['Body'])
rawdata['itemid'] = rawdata['itemid'].astype("int64")
rawdata['hadm_id'] = rawdata['hadm_id'].astype("int64")

In [3]:
rawdata.shape

(64914520, 6)

In [4]:
unique_hadm_by_year = rawdata.groupby('chartyear')['hadm_id'].nunique()
cumulative_sum = unique_hadm_by_year.cumsum()
total_sum = cumulative_sum.iloc[-1]
split_index = cumulative_sum[cumulative_sum >= 0.8 * total_sum].index[0]
print("split year is",split_index)

split year is 2178


In [5]:
hadm_id_indices = rawdata.groupby('hadm_id').apply(lambda group: {'start_index': group.index.min(), 'end_index': group.index.max()+1}).to_dict()
Labitemlist = rawdata['itemid'].value_counts().sort_values(ascending=False)
Labitemlist = Labitemlist.reset_index()
Labitemlist.columns = ['unique_value', 'count']
len(Labitemlist)

375

In [16]:
Labitemlist.iloc[10,:]

unique_value      51222
count           1901549
Name: 10, dtype: int64

In [6]:
itemid = Labitemlist['unique_value'][4]
print(itemid)
this_rl = rawdata[rawdata['itemid']==itemid][['hadm_id','chartyear','valuenum','charttime_diff_hours']]
this_rl.shape

50912


(1995732, 4)

In [7]:
rawdata.head()

,hadm_id,itemid,label,chartyear,charttime_diff_hours,valuenum
0,20000019,51222,Hemoglobin,2159,0,9.2
1,20000019,50868,Anion Gap,2159,0,14.0
2,20000019,51248,MCH,2159,0,29.8
3,20000019,50971,Potassium,2159,0,3.5
4,20000019,51265,Platelet Count,2159,0,183.0


In [8]:
selected_rl = pd.DataFrame(columns=range(len(Labitemlist)*2+4), index=range(len(this_rl)))
i = 4
for name in Labitemlist['unique_value']:
    selected_rl.rename(columns={i: name}, inplace=True)
    i=i+1
    selected_rl.rename(columns={i: f"{name}_dt"}, inplace=True)
    i=i+1
selected_rl.rename(columns={0: 'hadm_id'}, inplace=True)
selected_rl.rename(columns={1: 'chartyear'}, inplace=True)
selected_rl.rename(columns={2: 'valuenum'}, inplace=True)
selected_rl.rename(columns={3: 'charttime_diff_hours'}, inplace=True)
selected_rl = selected_rl.set_index(this_rl.index)
selected_rl['hadm_id'] = this_rl['hadm_id']
selected_rl['chartyear'] = this_rl['chartyear']
selected_rl['valuenum'] = this_rl['valuenum']
selected_rl['charttime_diff_hours'] = this_rl['charttime_diff_hours']
selected_rl.shape

(1995732, 754)

In [13]:
selected_rl.head()

,hadm_id,chartyear,valuenum,charttime_diff_hours,50971,50971_dt,50983,50983_dt,50902,50902_dt,...,51088,51088_dt,51795,51795_dt,51068,51068_dt,51105,51105_dt,50881,50881_dt
9,20000019,2159,1.1,0,3.5,0.0,137.0,0.0,103.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,20000019,2159,1.2,24,3.7,0.0,136.0,0.0,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,20000019,2159,0.9,49,3.9,0.0,138.0,0.0,102.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,20000024,2151,1.1,0,5.2,0.0,140.0,0.0,104.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,20000034,2174,2.3,0,4.8,0.0,141.0,0.0,106.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print("Current time:", datetime.now())

for inx, row in selected_rl.iterrows():
  hadm_id = row['hadm_id']
  start_index = hadm_id_indices.get(hadm_id)['start_index']
  # Get the end index for selection
  end_index = hadm_id_indices.get(hadm_id)['end_index']
  filtered_df = pd.concat([rawdata.iloc[start_index:inx], rawdata.iloc[inx + 1:end_index]])
  filtered_df['abs_time_diff'] = abs(filtered_df['charttime_diff_hours']-row['charttime_diff_hours'])
  idx_min_abs_time_diff = filtered_df.groupby('itemid')['abs_time_diff'].idxmin()
  pred_df = filtered_df.loc[idx_min_abs_time_diff, ['itemid', 'valuenum','abs_time_diff']]
  pred_df['itemid'] = pred_df['itemid'].astype("int64")
  for _,prow in pred_df.iterrows():
    iid = prow['itemid'].astype("int64")
    selected_rl.at[inx,iid] = prow['valuenum']
    selected_rl.at[inx,f"{iid}_dt"] = prow['abs_time_diff']
print("Current time:", datetime.now())


Current time: 2024-05-13 20:49:32.124962


In [ ]:
file_name = f"alllabs1000adm_{itemid}.csv"
file_key = f"Labrado/alllabs1000adm/alllabs1000adm_{itemid}.csv"
s3.upload_file(file_name, bucket_name,file_key)

In [ ]:
selected_rl.to_csv(f"alllabs1000adm_{itemid}.csv")